In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 01 01:11:02 2016
@author: DIP
"""

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [2]:
import re
import os

import pandas as pd
import numpy as np
import string
from nltk.tokenize import sent_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
  
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text



In [3]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^.a-zA-z0-9\s]' if remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern,'', text)
    return text



def normalize_corpus(corpus, contraction_expansion = True, text_lower_case = True, special_char_removal = True,remove_digits=True):

    normalized_corpus = []
    for doc in corpus:
        if text_lower_case:
            doc = doc.lower()

        if special_char_removal:
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits = remove_digits)
        doc = re.sub(' +',' ', doc)
        if contraction_expansion:
            doc = expand_contractions(doc)

        normalized_corpus.append(doc)
    return normalized_corpus

directory = r'C:\Users\amall\OneDrive\Desktop\project\stories'
#text_lst = []# summary_lst = []
t_lst = []
s_lst = []
for file in os.listdir(directory): #t_lst = []# s_lst = []
    data=open(os.path.join(directory,file), encoding = 'utf-8', errors='ignore')
    
    x = [line.strip('\n').replace("\n"," ") for line in re.split(r'@highlight', data.read() ,re.M)]
    
    summary = x[1: ]
    text = x[0]
   
    summary_str = '.'
    t_lst.append(text)
    s_lst.append(summary_str.join(summary))
    
    

news_df = pd.DataFrame(list(zip(s_lst, t_lst)), columns = ['Summary', 'Text'])
#print(news_df.head)

news_df2 = pd.DataFrame(list(zip(normalize_corpus(news_df['Summary']),normalize_corpus(news_df['Text']))),columns = ['clean_sum','clean_text'])
#print(news_df2.head)
#print(news_df2['clean_sum'])
#news_df2.to_csv('news.csv', index=False, encoding='utf-8')

#print(sent_tokenize(news_df2['clean_sum'][20]))
#print(len(sent_tokenize(news_df2['clean_sum'][20])))

#sent_count = []
#for index,row in news_df2.iterrows():
    #sent_count.append(len(sent_tokenize(row[0])))



In [4]:
#sentence tokenization
List=[]
sum1=[]
finalsum=[]
corp= news_df2['clean_text']
for index,row in news_df2.iterrows():
    #print(row[1])
    tsentences=sent_tokenize(row[1])
    List.append(tsentences)
    print(type(tsentences))



<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [12]:
#finding word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

#len(word_embeddings)
print(type(word_embeddings))


<class 'dict'>


In [11]:
sentence_vectors = []
#finding the sentence vectors
for i in List:
    for j in i:
        if len(j) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in j.split()])/(len(j.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
        sentence_vectors[0].shape

#similarity matrix
    sim_mat = np.zeros([len(i), len(i)])
    for k in range(len(i)):
        for l in range(len(i)):
            if i != j:
                sim_mat[k][l] = cosine_similarity(sentence_vectors[k].reshape(1,100), sentence_vectors[l].reshape(1,100))[0,0]

    sim_mat[0][1]
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
#ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    ranked_sentences=sorted(((score,index) for index,score in scores.items()),reverse=True)
    
    top_sentence_indices=[ranked_sentences[index][1] for index in range(3)]
    top_sentence_indices.sort()
    str=(" ".join(np.array(i)[top_sentence_indices]))
    #sum1.append(str);
    print(str);
    print("**********")
    #finalsum.append(sum1);
    #finalsum.append('/nnnnnnnn')
    
    
#print(finalsum)
print(news_df2['clean_sum'])    
    


the country land and business association and the countryside alliance claim the ban will make it tougher to control rats in parks and urban wasteland and say the hse is going further than it should . complaints the country land and business association and the countryside alliance claim the ban will make it tougher to control rats in parks and urban wasteland and say the hse is going further than it should the products are used in wider countryside areas by around 75 per cent of gamekeepers according to the national gamekeepers organisation . but a health and safety executive spokesman insisted that the poisons should be restricted to in and around buildings .
**********
but officials told the journal the systems broad reach makes it more likely that purely domestic communications will be incidentally intercepted and collected in the hunt for foreign ones . we defend the united states from such threats while fiercely working to protect the privacy rights of u . according to the journa

In [7]:
#for i in range(len(i)):
    #for j in range(len(i)):
    
    